In [1]:
from os import chdir
chdir('../')
%matplotlib inline
from lib import *

pd.options.display.precision = 4
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Lato'
mpl.rcParams['font.weight'] = 700
sns.set(font='Lato', font_scale=1)
sns.set()
np.random.seed(42)
suppress_warnings()

In [2]:
feature = pd.read_pickle('/home/jovyan/work/DSI/projects/project_3/data/feature')
target= pd.read_pickle('/home/jovyan/work/DSI/projects/project_3/data/target')

In [3]:
display(feature.shape)
display(target.shape)

(10768, 1000)

(10768, 1)

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_classif
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso, LogisticRegression,LinearRegression
from sklearn.pipeline import Pipeline

# Step 2 Identify Features

In [5]:
from sklearn.feature_selection import SelectKBest, \
                                      SelectFromModel, \
                                      RFE, SelectPercentile, \
                                      SelectFdr, SelectFpr
from sklearn.feature_selection import f_regression, f_classif

### Pipeline 1 - Select K Best - Logistic Regression

In [6]:
pipe_for_gs = Pipeline([
    ('skb', SelectKBest(score_func=f_classif)),
    ('lr', LogisticRegression(C=1E10,n_jobs=-1,random_state=42))
])

In [7]:
params = {
    'skb__k':[50,150,250,500],
}

In [8]:
gspipe = GridSearchCV(pipe_for_gs,
                      param_grid=params,
                      n_jobs=-1,
                      cv=ShuffleSplit(random_state=42))

In [9]:
gspipe.fit(feature[:220], np.ravel(target[:220].values))

GridSearchCV(cv=ShuffleSplit(n_splits=10, random_state=42, test_size=0.1, train_size=None),
       error_score='raise',
       estimator=Pipeline(steps=[('skb', SelectKBest(k=10, score_func=<function f_classif at 0x7f0c7d9f66a8>)), ('lr', LogisticRegression(C=10000000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'skb__k': [50, 150, 250, 500]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [10]:
gspipe.best_score_

0.40454545454545454

In [11]:
gspipe.best_estimator_.named_steps

{'lr': LogisticRegression(C=10000000000.0, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=42,
           solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
 'skb': SelectKBest(k=150, score_func=<function f_classif at 0x7f0c7d9f66a8>)}

In [12]:
gspipe_results = pd.DataFrame(gspipe.cv_results_)
gspipe_results.sort_values('rank_test_score', ascending=True).T

,1,2,3,0
mean_fit_time,0.0145,0.02174,0.0476,0.01637
mean_score_time,0.0004894,0.0004924,0.0005328,0.0004555
mean_test_score,0.4045,0.4045,0.4045,0.3773
mean_train_score,1,1,1,0.9384
param_skb__k,150,250,500,50
params,{'skb__k': 150},{'skb__k': 250},{'skb__k': 500},{'skb__k': 50}
rank_test_score,1,1,1,4
split0_test_score,0.3636,0.3636,0.4545,0.4545
split0_train_score,1,1,1,0.8889
split1_test_score,0.4091,0.4091,0.4091,0.5


In [13]:
gspipe_results['mean_fit_time'].sum()

0.10021893978118897

In [14]:
### Pipeline1 for .1% (220 rows)
print("Fit time .1% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score .1% data: ",gspipe.best_score_)
picked_pipeline_220row = \
        pd.DataFrame.to_pickle(gspipe_results,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline1_lr_pt1_percent')

Fit time .1% data:  0.100218939781
Best Score .1% data:  0.404545454545


In [15]:
### Pipeline1 for .5% (1100 rows)

gspipe.fit(feature[:1100], np.ravel(target[:1100].values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time .5% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score .5% data: ",gspipe.best_score_)

gspipe_results_1100 = pd.DataFrame(gspipe.cv_results_)
picked_Pipeline1_1100row = \
        pd.DataFrame.to_pickle(gspipe_results_1100,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline1_lr_pt5_percent')

Fit time .5% data:  1.59021253586
Best Score .5% data:  0.500909090909


In [16]:
### Pipeline1 for 1% (2200 rows)

gspipe.fit(feature[:2200], np.ravel(target[:2200].values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time 1% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score 1% data: ",gspipe.best_score_)

gspipe_results_2200 = pd.DataFrame(gspipe.cv_results_)
picked_Pipeline1_2200row = \
        pd.DataFrame.to_pickle(gspipe_results_2200,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline1_lr_1_percent')

Fit time 1% data:  1.37021312714
Best Score 1% data:  0.485909090909


In [17]:
### Bench Marking for 5% (11000 rows)

gspipe.fit(feature, np.ravel(target.values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time 5% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score 5% data:",gspipe.best_score_)
gspipe_results_11000 = pd.DataFrame(gspipe.cv_results_)
picked_GS_BenchMarkig_11000row = \
        pd.DataFrame.to_pickle(gspipe_results_11000,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline1_lr_5_percent')

Fit time 5% data:  5.4187497139
Best Score 5% data: 0.515970287837
